# CƠ CHẾ HOẠT ĐỘNG CỦA PIPELINE

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
) 

cho output là :


In [ ]:
[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]

PIPELINE gồm 3 bước : tiền xử lý, đưa các đầu vào qua mô  hình và hậu xử lý.

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline-dark.svg)

## BƯỚC 1: Tiền xử lý với một tokenizer

Đầu tiên chuyển đầu vào văn bản thành dạng số để mô hình hiểu, để làm điều này ta dùng tokenizer:

        - Tách đầu vào thành các từ, từ phụ, hoặc ký hiệu đc gọi là tokens.
        - Ánh xạ mỗi token thành một số nguyên
        -Thêm đầu vào bổ sung có thể hữu ích cho mô hình.

Ta cần xử lý giống như khi mô hình đc huấn luyện trước, có thể sử dụng lớp AutoTokenizer và phương thức from_pretrained(). Sử dụng checkpoint của mô hình , nó sẽ tự tìm nạp dữ liệu được liên kết với tokenizer của mô hình và lưu cache.(chỉ tải ở lần chạy đầu tiên).

Checkpoint mặc định của sentiment-analysis là distilbert-base-unsased-finetuned-....

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Khi có tokenizer, ta có thể truyển trực tiếp các câu của mình  vào bên trong và nhận lại một từ điển đã sẵn sàng để cung cấp cho mô hình. Chuyển các ID đầu vào thành tensor.

Các mô hình transformers chỉ nhân tensor là đầu vào

Để chỉ định loại tensor trả về , dùng return_tensors:

In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

Bạn có thể chuyển một câ thành danh sách các câu nhỏ, chỉ định các loại tensors mình muốn nếu ko nó sẽ trả kết quả là một danh sách
 
 
tensor Pytorch:

In [ ]:
{
    'input_ids': tensor([
        [  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172, 2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,     0,     0,     0,     0,     0,     0]
    ]), 
    'attention_mask': tensor([
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ])
}

## Tiếp theo là cho đầu ra đi qua Models

Tải xuống mô hình đc huấn luyện trước.


In [ ]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Kiến trúc này chứa modun Transformers cơ sở: với một số đầu vào nó xuất ra hidden states(đặc trưng). Mỗi đầu vào ta sẽ truy xuất một vector đa chiều đại diện cho sự hiểu theo ngữ cảnh của đầu vào đó bằng mô hình Transformers.




### Vector đa chiều


Đầu ra vector của modun Transformers thường lớn với 3 chiều:

    - Batch size: Số chuỗi được xử lý tại 1 thời điểm.
    - Độ dài chuỗi(shape): Độ dìa biểu diễn số vector của chuỗi.
    - Kích thước ấn: Kích thước vector của mỗi đầu vào mô hình.

Kích thước ẩn có thể rất lớn: 768 hoặc các mô hình lớn có thể đạt tới 3072)


### Đầu mô hình

Lấy vector đa chiều của các trạng thái ẩn làm đầu vào và chiếu chúng lên 1 chiều khác. Nó thường bao gồm 1 hoặc nhiều lớp tuyển tính.


![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head-dark.svg)

Đầu ra của mô hình Transformer được gửi trực tiếp đến đầu mô hình để được xử lý.

Ở trên, mô hình được biểu diễn bằng lớp nhúng của nó và các Layer tiếp theo. Lớp Embedding chuyển đổi mỗi ID trong đầu vào được mã hóa thành một vector đại diện cho token được liên kết. Các lớp tiếp theo thao tác với vector bằng cách sử dụng cơ chế Atten để tạo ra biểu diễn cuối cùng của các câu.

Các kiến trúc trong Transformers:

    - *Model (truy xuất các trạng thái ẩn)
    - *ForCausalLM
    - *ForMaskedLM
    - *ForMultipleChoice
    - *ForQuestionAnswering
    - *ForSequenceClassification
    - *ForTokenClassification
and others 🤗

Với ví dụ của mình, chúng ta sẽ cần một mô hình có đầu phân loại tuần tự (để có thể phân loại các câu là khẳng định hoặc phủ định). Vì vậy, ta sẽ không sử dụng lớp AutoModel mà là AutoModelForSequenceClassification:



In [ ]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

Đầu mô hình lấy các vector đa chiều mà chúng ta thấy trước đây và xuất ra các vector có chứa 2 giá trị(mỗi giá trị là 1 nhãn)

## Hậu xử lý

In [ ]:
print(outputs.logits.shape)

#ket qua
 
 
torch.Size([2, 2])

Vì chúng ta chỉ có hai câu và hai nhãn, kết quả nhận được từ mô hình của chúng ta là dạng 2 x 2.

